In [1]:
import pandas as pd
import numpy as np
from statsmodels.sandbox.regression.gmm import GMM
from scipy.optimize import minimize

In [2]:
# Load the dataset
data = pd.read_csv("/MBAN- Schulich/MBAN- Sem 2/Predictive Modeling/Mid term Project/midterm_partone.csv")

In [3]:
# Display the first few rows to understand its structure
print(data.head())

   Constant  Stock Change  Inventory Turnover  Operating Profit  \
0         1      0.870332            1.795946          0.115846   
1         1     -0.047347            1.395501          0.436967   
2         1      0.001176            1.664563          0.541016   
3         1     -0.901200            1.605738          0.539399   
4         1     -0.176353            1.591451          0.539938   

   Interaction Effect  Current Ratio  Quick Ratio  Debt Asset Ratio  
0            0.208053       1.672527     0.255171          0.473317  
1            0.609788       1.637261     0.221763          0.489967  
2            0.900555       1.640619     0.189141          0.374269  
3            0.866133       1.436221     0.131944          0.224399  
4            0.859285       1.433140     0.183095          0.213446  


In [4]:
# Define dependent, independent, and instrumental variables
y_vals = np.array(data["Stock Change"])  # Dependent variable
x_vals = np.array(data[["Inventory Turnover", "Operating Profit", "Interaction Effect"]])  # Endogenous variables
iv_vals = np.array(data[["Current Ratio", "Quick Ratio", "Debt Asset Ratio"]])  # Instrumental variables

In [22]:
# Define the custom GMM class to handle the moment conditions with delta
class GMMWithBias:
    def __init__(self, y_vals, x_vals, iv_vals):
        self.y_vals = y_vals
        self.x_vals = x_vals
        self.iv_vals = iv_vals

    # Define the moment conditions with optional data arguments
    def moment_conditions(self, params, y_vals=None, x_vals=None, iv_vals=None):
        if y_vals is None:
            y_vals = self.y_vals
        if x_vals is None:
            x_vals = self.x_vals
        if iv_vals is None:
            iv_vals = self.iv_vals

        # Unpack parameters (coefficients and delta)
        p0, p1, p2, p3, delta = params
        # Calculate residuals with delta
        residuals = y_vals - (p0 + p1 * x_vals[:, 0] + p2 * x_vals[:, 1] + p3 * x_vals[:, 2]) - delta
        
        # Moment conditions with residuals and instruments
        g = np.column_stack((residuals,
                             residuals * x_vals[:, 0],
                             residuals * x_vals[:, 1],
                             residuals * x_vals[:, 2],
                             residuals * iv_vals[:, 0],
                             residuals * iv_vals[:, 1],
                             residuals * iv_vals[:, 2]))
        return g

    # Define the GMM objective function to minimize
    def gmm_objective(self, params, y_vals=None, x_vals=None, iv_vals=None):
        moments = self.moment_conditions(params, y_vals, x_vals, iv_vals)
        return np.sum(moments ** 2)

In [23]:
# Initialize parameters
initial_params = np.array([0.1, 0.1, 0.1, 0.1, 0.1])

In [24]:
# Fit the GMM model
gmm_model = GMMWithBias(y_vals, x_vals, iv_vals)

In [25]:
# Optimize to find the best-fit parameters
result = minimize(gmm_model.gmm_objective, initial_params)

In [26]:
# Extract and print the estimated parameters
B_estimated = result.x[:-1]  # Estimated coefficients for x_vals
delta_estimated = result.x[-1]  # Estimated bias term delta

In [27]:
# Display the results
print("Estimated Coefficients (B):", B_estimated)
print("Estimated Bias Term (Delta):", delta_estimated)
print("Optimization Success:", result.success)
print("Objective Function Value:", result.fun)

Estimated Coefficients (B): [-0.02226153  0.00066278 -0.03505035  0.00049336]
Estimated Bias Term (Delta): -0.022261532077181536
Optimization Success: False
Objective Function Value: 1250539.0814367211


In [28]:
# Bootstrap for standard errors and confidence intervals
bootstrap_samples = 1000
delta_estimates = []

In [29]:
from sklearn.utils import resample

for _ in range(bootstrap_samples):
    y_sample, x_sample, iv_sample = resample(y_vals, x_vals, iv_vals, random_state=None)
    result_boot = minimize(gmm_model.gmm_objective, initial_params, args=(y_sample, x_sample, iv_sample))
    if result_boot.success:
        delta_estimates.append(result_boot.x[-1])

In [30]:
# Calculate statistics for delta
delta_std_error = np.std(delta_estimates)
delta_confidence_interval = (np.percentile(delta_estimates, 2.5), np.percentile(delta_estimates, 97.5))

In [31]:
print("Standard Error of Delta:", delta_std_error)
print("95% Confidence Interval for Delta:", delta_confidence_interval)

Standard Error of Delta: 0.044055179831660365
95% Confidence Interval for Delta: (np.float64(-0.08561923067701317), np.float64(0.06488949197560152))
